# Automatic feature selection with LASSO regression

In this notebook we will learn how LASSO (Least Absolute Shrinkage and Selection Operator) regression works and how it can assist in automatically selecting which variables should be included using a **Cross-Validation** perspective.

### Start by importing packages

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from sklearn.datasets import load_diabetes
from sklearn.feature_selection import SelectFromModel
from sklearn import datasets, linear_model
from sklearn.linear_model import LassoCV, Lasso
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

import statsmodels
from statsmodels.api import OLS

#### Load dataset and inspect it

-   Again we're going to use our diabetes dataset.
    -   Inspect it again just to remind yourself what is in it.

In [ ]:
# Load the diabetes dataset
diabetes = load_diabetes()

# Extract numpy arrays for X and y
X = diabetes.data
y = diabetes.target

# Get the feature names
feature_names = diabetes.feature_names

# Print them out to look at them
print(diabetes['DESCR'])
print(feature_names)

### Select subset of data

-   To speed up calculation, we're going to just use the first 150 observations
    -   Use numpy slice notation to grab them out of the X, y

In [ ]:
# Remember the notation for slices: X[start:stop:step]
# where omitted values default to start=0, stop=size of dimension, step=1
# So, X[:150] means "grab the first 150 rows"
# X = X[:]
# y = y[:150]

# Split the data into training/testing sets using scikit-learn's train_test_split() function
from sklearn.model_selection import train_test_split

# Split the data into training/testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### Run OLS first (for comparison)

Remember the standard Sklearn model steps:

1.  create the model object
2.  call the object's fit method.
3.  use the fitted model to predict something.
4.  assess the predictions.

In [ ]:
# Create linear regression object
model_ols = linear_model.LinearRegression()

# Train the model using the training sets
model_ols.fit(X_train, y_train)

# Make predictions using the testing set
y_train_hat = model_ols.predict(X_train)

# The coefficients
print("Coefficients: \n", model_ols.coef_)

# The mean squared error
mse_ols = mean_squared_error(y_train, y_train_hat)
print("Mean squared error:", mse_ols)

# The coefficient of determination: 1 is perfect prediction
r2_train_ols = r2_score(y_train, y_train_hat)
print("Coefficient of determination:", r2_train_ols)

### Do it again in the econometrics style

-   Recall that the package statsmodels is closer to the econometrician's way of doing things.
-   We're going to quickly repeat the steps above but with Statsmodels so we can view it in a nice table form.

In [ ]:
x_train_with_constant = statsmodels.api.add_constant(X_train)
result = OLS(y_train, x_train_with_constant).fit().summary()

print(result)

# Notice that the R-squared is the same as above

### Plot y and y_hat

-   Let's also plot y and y_hat compared to one of the most important variables, BMI.
    -   We'll see both y and y_hat resemble each other.

In [ ]:
# Plot outputs (comparing 1 variable (BMI in column 3) to y and y_hat
# plt.scatter() takes 2 numpy arrays.
# The X[:, 3] notation gets all rows but only in column at index 3 (BMI)
# And recall that y is "a quantitative measure of disease progression one year after baseline"

# Plot the actual y values in black
plt.scatter(X_train[:, 3], y_train, color="black")

# Plot the predicted y values in blue
plt.scatter(X_train[:, 3], y_train_hat, color="blue")

# Add vertical axis label
plt.ylabel('Disease progression')

# Add horizontal axis label
plt.xlabel('BMI')

# DISCUSSION QUESTION: Why is BMI centered on 0? Isn't it like a value of 20-30?

## Switch to LASSO

Now that we've spent all this time setting up our python environment and getting sklearn, it's almost a trivial step in many cases to try out the latest-and-greatest model.

### Create a LASSO model object

-   Today's goal, however, is to do Lasso on this same dataset.
    -   To start, lets create a Lasso object.
    -   Notice that we are not setting the alpha/gamma value when we create it.
-   STEP 1: Create the object

In [ ]:
# Create an empty Lasso object with a few initial parameters set
model_lasso = Lasso(alpha=1.0, random_state=42, max_iter=10000) # Note, alpha is set by default to 1.0 so we could have omitted it here (though I kept it in to make it clear)
print(model_lasso)

### Fit the LASSO

-   STEP 2: Call the lasso.fit() method.
    -   The key think is to understand which X and y you are working with
    -   We haven't split into training and testing YET, so it doesn't matter here

In [ ]:
y_train_hat_lasso = model_lasso.fit(X_train, y_train)
print(model_lasso, ': I might look the same but now i\'m FIT!')

### Predict with the LASSO

-   STEP 3: Predict
    -   Save the results to a new variable called y_hat_lasso
        -   We will use this later

In [ ]:
y_train_hat_lasso = model_lasso.predict(X_train)
print('y_hat_lasso', y_train_hat_lasso)

### Plot it too to compare it with the OLS plot from above

What do you see. Is this expected?

In [ ]:
# Plot outputs
plt.scatter(X_train[:, 3], y_train, color="black")
plt.scatter(X_train[:, 3], y_train_hat_lasso, color="blue")

plt.show()

### Compare the actual coefficients created

Class question: How are they different? And how are they similar?

In [ ]:
print(model_lasso.coef_)
print(model_ols.coef_)

## Exercise 1

Use a loop to identify the best value of alpha, as measured by r-squared.

Write all of the alphas and associated r2 into a dictionary

Discussion question for once you're done: what was the optimal alpha and why does this make sense? How does this compare to OLS? Why is it that way?

In [ ]:
# Exercise 1 Code
scores = {}
alphas = np.logspace(-5, -0.05, 30)
for alpha in alphas:
    model_lasso = Lasso(alpha=alpha, random_state=0, max_iter=10000)
    model_lasso.fit(X_train, y_train)
    y_train_hat_lasso = model_lasso.predict(X_train)
    r2 = r2_score(y_train, y_train_hat_lasso)
    scores[alpha] = r2
    print('Alpha', alpha, 'r2', r2)

# Quick way to get the value from the highest-valued dictionary entry
best_alpha = max(scores, key=scores.get)

# print best_alpha with all the significant digits
print('best_alpha', best_alpha)

# This is not very pretty. There MUST be a better way to do this.
# There is!

## Operationalizing CV with GridSearch

-   It seems a little weird to be automatically finding the best model.

    -   If we were just applying this to the dataset a single time, this would indeed be p-hacking to the extreme.

    -   However, showing its performance on UNSEEN data is quite the opposite of p-hacking.

-   Scikit Learn lets us operationalize our method for finding the best model by using GridSearch.

-   GridSEarch will search for the best set of "hyperparameters" we can find.

    -   Here we only have 1 hyperparameter, alpha.

        -   Our plan will be to split the training data into 5 folds, and then test each alpha on each fold, keeping track of which alpha performed best.

            -   Why split into folds instead of just testing each alpha on one split of the training data?

                -   We can eek-out much more training power by reusing the data lots of times in different splits.

![](images/paste-1.png)

### Setup the inputs for GridSearchCV

-   Define a range of alphas we will test using numpy logspace:

In [ ]:
alphas = np.logspace(-6, -0.5, 30)
print(alphas)

-   We are going to be passing this range of tuning parameters to a GridSearch

    -   It .fit() function will test which alpha works best across all of the folds.

-   First though, we have to put the alphas into the form the GridSearchCV function Expects, which is a list of dictionaries.

    -   More complex models might have lots of different hyperparameters.

In [ ]:
tuning_parameters = [{'alpha': alphas}]

### Choose how many folds to use

-   Recall that CV works by calculating the fit quality of different folds of the training data.

    -   Here we will just use 5 folds. GridSearchCV will automatically implement the folding and testing logic.

In [ ]:
n_folds = 5

#### Create the lasso_cv object from the lasso object

-   Finally, we have all our objects ready to pass to the GridSearchVC function

    -   Create a new GridSearchCV object.

        -   Notice that we are reusing model_lasso object we created above.

In [ ]:
model_lasso_cv = GridSearchCV(model_lasso, tuning_parameters, cv=n_folds, refit=False)

#### Fit the lasso_cv object

When we call the model_lasso_cv.fit() method, we will iteratively be calling the Lasso.fit() with different permutations of tuned parameters and then will return the classifier with the best CV fit.

In [ ]:
model_lasso_cv.fit(X_train, y_train)

-   Although it was just one line, we just ran a BOATLOAD of Lasso regressions.

-   The `model_lasso_cv` object now has a variety of diagnostic metrics, reporting back on different folds within the Cross Validation. Take a look at them below.

In [ ]:
print('model_lasso_cv keys returned:', model_lasso_cv.cv_results_.keys())

### Extract results by score

Some relevant results are as below, which we'll extract and assign to lists.

In [ ]:
scores = model_lasso_cv.cv_results_['mean_test_score']
scores_std = model_lasso_cv.cv_results_['std_test_score']

print('scores', scores)
print('scores_std', scores_std)

## Exercise 2:

With your table, explore the scores and alphas lists we've created. Identify which alpha is the best, based on the MSE score returned. A challenge here is that sklearn gave us the scores as a list rather than a dictionary (as we built above), so you will need to use the list to create the dictionary.

One way to consider doing this would be to create a for loop to iterate through a range(len(scores)): object, saving the alphas and scores to a new dictionary, as in the starter code below.

Save the optimal alpha as a new variable called chosen_alpha.

In [ ]:
# Exercise 2 Code

output_dict = {}
for i in range(len(scores)):
    output_dict[alphas[i]] = scores[i]
    
best_alpha = max(output_dict, key=output_dict.get)

print('best_alpha', best_alpha)

### Plot out the alphas and their scores

In [ ]:
plt.figure().set_size_inches(8, 6)
plt.semilogx(alphas, scores)
# Also add an annotation vertical dotted line indicating the optimal alpha
plt.axvline(best_alpha, linestyle='--', color='.5')

### Use the built-in attributes to get the best alpha

-   Fortunately, the authors provide a useful best_params\_ attribute.

In [ ]:
print('best_parameters:', model_lasso_cv.best_params_)

-   Extract the best alpha, which we will use later.

In [ ]:
chosen_alpha = model_lasso_cv.best_params_['alpha']
print('chosen_alpha', chosen_alpha)

### Rerun LASSO with the best alpha

-   Now we can rerun a vanilla (no CV) version of Lasso with that specific alpha.

    -   This will return, for instance, a `model_lasso_cv_2.coef_` list.

In [ ]:
model_lasso_cv_2 = Lasso(alpha=chosen_alpha, random_state=0, max_iter=10000).fit(X_train, y_train)

print("coefficients", model_lasso_cv_2.coef_)

-   Simply looking at the coefficients tells us which are to be included. Question: How will we know just by looking?

### Extract the feature names and column indices of the features that Lasso has selected.

In [ ]:
selected_coefficient_labels = []
selected_coefficient_indices = []
for i in range(len(model_lasso_cv_2.coef_)):
    print('Coefficient', feature_names[i], 'was', model_lasso_cv_2.coef_[i])
    if abs(model_lasso_cv_2.coef_[i]) > 0:
        selected_coefficient_labels.append(feature_names[i])
        selected_coefficient_indices.append(i)

This process led us to the following selected_coefficient_labels:

In [ ]:
print('selected_coefficient_labels', selected_coefficient_labels)

## Switch back to slides

## Post-LASSO

Finally, now that we have our selected labels, we can use them to select the numpy array columns that we want to use for a post-LASSO run.

In [ ]:
X_train_selected = X_train[:, selected_coefficient_indices]
print('X_train_selected', X_train_selected)

Plug this new x matrix into our statsmodels OLS function and print that out.

In [ ]:
# Create the model and fit it using scikit learn linear_model
ols_model_selected = linear_model.LinearRegression()
ols_model_selected.fit(X_train_selected, y_train)

# Make predictions using the testing set
y_train_hat = ols_model_selected.predict(X_train_selected)

# report the r-sqr
r2_train_ols = r2_score(y_train, y_train_hat)
print('r2_train_ols', r2_train_ols)